# Skip-gram Word2Vec

In this notebook, I'll lead you through using PyTorch to implement the [Word2Vec algorithm](https://en.wikipedia.org/wiki/Word2vec) using the skip-gram architecture. By implementing this, you'll learn about embedding words for use in natural language processing. This will come in handy when dealing with things like machine translation.

## Readings

Here are the resources I used to build this notebook. I suggest reading these either beforehand or while you're working on this material.

* A really good [conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) of Word2Vec from Chris McCormick 
* [First Word2Vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
* [Neural Information Processing Systems, paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for Word2Vec also from Mikolov et al.

---
## Word embeddings

When you're dealing with words in text, you end up with tens of thousands of word classes to analyze; one for each word in a vocabulary. Trying to one-hot encode these words is massively inefficient because most values in a one-hot vector will be set to zero. So, the matrix multiplication that happens in between a one-hot input vector and a first, hidden layer will result in mostly zero-valued hidden outputs.

<img src='assets/one_hot_encoding.png' width=50%>

To solve this problem and greatly increase the efficiency of our networks, we use what are called **embeddings**. Embeddings are just a fully connected layer like you've seen before. We call this layer the embedding layer and the weights are embedding weights. We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix. We can do this because the multiplication of a one-hot encoded vector with a matrix returns the row of the matrix corresponding the index of the "on" input unit.

<img src='assets/lookup_matrix.png' width=50%>

Instead of doing the matrix multiplication, we use the weight matrix as a lookup table. We encode the words as integers, for example "heart" is encoded as 958, "mind" as 18094. Then to get hidden layer values for "heart", you just take the 958th row of the embedding matrix. This process is called an **embedding lookup** and the number of hidden units is the **embedding dimension**.

<img src='assets/tokenize_lookup.png' width=50%>
 
There is nothing magical going on here. The embedding lookup table is just a weight matrix. The embedding layer is just a hidden layer. The lookup is just a shortcut for the matrix multiplication. The lookup table is trained just like any weight matrix.

Embeddings aren't only used for words of course. You can use them for any model where you have a massive number of classes. A particular type of model called **Word2Vec** uses the embedding layer to find vector representations of words that contain semantic meaning.

---
## Word2Vec

The Word2Vec algorithm finds much more efficient representations by finding vectors that represent the words. These vectors also contain semantic information about the words.

<img src="assets/context_drink.png" width=40%>

Words that show up in similar **contexts**, such as "coffee", "tea", and "water" will have vectors near each other. Different words will be further away from one another, and relationships can be represented by distance in vector space.

<img src="assets/vector_distance.png" width=40%>


There are two architectures for implementing Word2Vec:
>* CBOW (Continuous Bag-Of-Words) and 
* Skip-gram

<img src="assets/word2vec_architectures.png" width=60%>

In this implementation, we'll be using the **skip-gram architecture** because it performs better than CBOW. Here, we pass in a word and try to predict the words surrounding it in the text. In this way, we can train the network to learn representations for words that show up in similar contexts.

---
## Loading Data

Next, we'll ask you to load in data and place it in the `data` directory

1. Load the [text8 dataset](https://s3.amazonaws.com/video.udacity-data.com/topher/2018/October/5bbe6499_text8/text8.zip); a file of cleaned up *Wikipedia article text* from Matt Mahoney. 
2. Place that data in the `data` folder in the home directory.
3. Then you can extract it and delete the archive, zip file to save storage space.

After following these steps, you should have one file in your data directory: `data/text8`.

In [1]:
# read in the extracted text file      
with open('data/text8') as f:
    text = f.read()

# print out the first 100 characters
print(text[:100])

 anarchism originated as a term of abuse first used against early working class radicals including t


## Pre-processing

Here I'm fixing up the text to make training easier. This comes from the `utils.py` file. The `preprocess` function does a few things:
>* It converts any punctuation into tokens, so a period is changed to ` <PERIOD> `. In this data set, there aren't any periods, but it will help in other NLP problems. 
* It removes all words that show up five or *fewer* times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. 
* It returns a list of words in the text.

This may take a few seconds to run, since our text file is quite large. If you want to write your own functions for this stuff, go for it!

In [2]:
import utils

# get list of words
words = utils.preprocess(text)
print(words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


In [3]:
# print some stats about this word data
print("Total words in text: {}".format(len(words)))
print("Unique words: {}".format(len(set(words)))) # `set` removes any duplicate words

Total words in text: 16680599
Unique words: 63641


### Dictionaries

Next, I'm creating two dictionaries to convert words to integers and back again (integers to words). This is again done with a function in the `utils.py` file. `create_lookup_tables` takes in a list of words in a text and returns two dictionaries.
>* The integers are assigned in descending frequency order, so the most frequent word ("the") is given the integer 0 and the next most frequent is 1, and so on. 

Once we have our dictionaries, the words are converted to integers and stored in the list `int_words`.

In [4]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

print(int_words[:30])

[5233, 3080, 11, 5, 194, 1, 3133, 45, 58, 155, 127, 741, 476, 10571, 133, 0, 27349, 1, 0, 102, 854, 2, 0, 15067, 58112, 1, 0, 150, 854, 3580]


In [5]:
int_to_vocab[0]

'the'

## Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

$$ P(0) = 1 - \sqrt{\frac{1*10^{-5}}{1*10^6/16*10^6}} = 0.98735 $$

I'm going to leave this up to you as an exercise. Check out my solution to see how I did it.

> **Exercise:** Implement subsampling for the words in `int_words`. That is, go through `int_words` and discard each word given the probablility $P(w_i)$ shown above. Note that $P(w_i)$ is the probability that a word is discarded. Assign the subsampled data to `train_words`.

In [6]:
from collections import Counter
import random
import numpy as np

threshold = 1e-5
word_counts = Counter(int_words)

#print(word_counts)
#print(list(word_counts.items()))  # dictionary of int_words, how many times they appear
# (Word as int, freq. of words)


total_words = len(int_words) # the total number of words


words_freq = {word: freq/total_words for word,freq in word_counts.items()}

# Calculate the words' probability using the above formula{the probability to remove the word}
# if the words_freq increases, then the probability of removing the words increasing 

words_probabilites = {word: 1 - np.sqrt(threshold/words_freq[word]) for word in word_counts.keys()}

# discard some frequent words, according to the subsampling equation
# create a new list of words for training

train_words = [word for word in int_words if random.random() > words_probabilites[word]]

print(train_words[:30])

[5233, 3080, 3133, 58, 155, 127, 10571, 27349, 15067, 58112, 10712, 214, 104, 454, 2731, 3672, 708, 539, 2757, 686, 7088, 44611, 2877, 5233, 602, 1134, 8983, 6437, 4186, 153]


In [7]:
# int_to_vocab[0]
total_words, len(train_words) # (16680599, 4628809)

(16680599, 4628568)

## Making batches

Now that our data is in good shape, we need to get it into the proper form to pass it into our network. With the skip-gram architecture, for each word in the text, we want to define a surrounding _context_ and grab all the words in a window around that word, with size $C$. 

From [Mikolov et al.](https://arxiv.org/pdf/1301.3781.pdf): 

"Since the more distant words are usually less related to the current word than those close to it, we give less weight to the distant words by sampling less from those words in our training examples... If we choose $C = 5$, for each training word we will select randomly a number $R$ in range $[ 1: C ]$, and then use $R$ words from history and $R$ words from the future of the current word as correct labels."

> **Exercise:** Implement a function `get_target` that receives a list of words, an index, and a window size, then returns a list of words in the window around the index. Make sure to use the algorithm described above, where you chose a random number of words to from the window.

Say, we have an input and we're interested in the idx=2 token, `741`: 
```
[5233, 58, 741, 10571, 27349, 0, 15067, 58112, 3580, 58, 10712]
```

For `R=2`, `get_target` should return a list of four values:
```
[5233, 58, 10571, 27349]
```

In [8]:
a = [0,1,2,3,4,5,6,7,8,9]
f = [0,1,2,3,4,5,6,7,8,9]



In [9]:
f.extend([a]*1)
#[batch_x]*len(batch_y)

In [10]:
f

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]

In [11]:
ran_num = random.randint(1, 5)

ran_num

1

In [12]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    # implement this function
    
    import random
    
    ran_num = random.randint(1, window_size)
    # print("The index is ", idx)
    # print("The random number is ", ran_num)
    
    bef = 0 if idx - ran_num < 0 else idx - ran_num
    aft = len(words) if idx + ran_num+1 > len(words) else idx + ran_num +1
    
    
    return list(words[bef:idx] + words[idx+1:aft])

In [13]:
# test your code!

# run this cell multiple times to check for random window selection
int_text = [i for i in range(10)]
print('Input: ', int_text)
idx=5 # word index of interest

target = get_target(int_text, idx=idx, window_size=5)
print('Target: ', target)  # you should get some indices around the idx

Input:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Target:  [2, 3, 4, 6, 7, 8]


### Generating Batches 

Here's a generator function that returns batches of input and target data for our model, using the `get_target` function from above. The idea is that it grabs `batch_size` words from a words list. Then for each of those batches, it gets the target words in a window.

In [14]:
import pdb 

def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            
            # pdb.set_trace()
            
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
            
            # pdb.set_trace()
            
        #print("x is ", x)
        #print("y is ", y)
        
        yield x, y

In [15]:
int_text = [i for i in range(20)]
x,y = next(get_batches(int_text, batch_size=4, window_size=5))

print('x\n', x)
print('y\n', y)

x
 [0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3]
y
 [1, 2, 3, 0, 2, 3, 0, 1, 3, 1, 2]


## Building the graph

Below is an approximate diagram of the general structure of our network.
<img src="assets/skip_gram_arch.png" width=60%>

>* The input words are passed in as batches of input word tokens. 
* This will go into a hidden layer of linear units (our embedding layer). 
* Then, finally into a softmax output layer. 

We'll use the softmax layer to make a prediction about the context words by sampling, as usual.

The idea here is to train the embedding layer weight matrix to find efficient representations for our words. We can discard the softmax layer because we don't really care about making predictions with this network. We just want the embedding matrix so we can use it in _other_ networks we build using this dataset.

---
## Validation

Here, I'm creating a function that will help us observe our model as it learns. We're going to choose a few common words and few uncommon words. Then, we'll print out the closest words to them using the cosine similarity: 

<img src="assets/two_vectors.png" width=30%>

$$
\mathrm{similarity} = \cos(\theta) = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|}
$$


We can encode the validation words as vectors $\vec{a}$ using the embedding table, then calculate the similarity with each word vector $\vec{b}$ in the embedding table. With the similarities, we can print out the validation words and words in our embedding table semantically similar to those words. It's a nice way to check that our embedding table is grouping together words with similar semantic meanings.

In [16]:
def cosine_similarity(embedding, valid_size=16, valid_window=100, device='cpu'):
    """ Returns the cosine similarity of validation words with words in the embedding matrix.
        Here, embedding should be a PyTorch embedding module.
    """
    
    # Here we're calculating the cosine similarity between some random words and 
    # our embedding vectors. With the similarities, we can look at what words are
    # close to our random words.
    
    # sim = (a . b) / |a||b|
    
    embed_vectors = embedding.weight
    
    # magnitude of embedding vectors, |b|
    magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
    
    # pick N words from our ranges (0,window) and (1000,1000+window). lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples,
                               random.sample(range(1000,1000+valid_window), valid_size//2))
    valid_examples = torch.LongTensor(valid_examples).to(device)
    
    valid_vectors = embedding(valid_examples)
    similarities = torch.mm(valid_vectors, embed_vectors.t())/magnitudes
        
    return valid_examples, similarities

## SkipGram model

Define and train the SkipGram model. 
> You'll need to define an [embedding layer](https://pytorch.org/docs/stable/nn.html#embedding) and a final, softmax output layer.

An Embedding layer takes in a number of inputs, importantly:
* **num_embeddings** – the size of the dictionary of embeddings, or how many rows you'll want in the embedding weight matrix
* **embedding_dim** – the size of each embedding vector; the embedding dimension

In [17]:
import torch
from torch import nn
import torch.optim as optim

In [18]:
class SkipGram(nn.Module):
    def __init__(self, n_vocab, n_embed):
        super().__init__()
        
        # complete this SkipGram model
        self.embed = nn.Embedding(n_vocab, n_embed)
        
        self.output = nn.Linear(n_embed,n_vocab)
        
        self.LogSof = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        
        # define the forward behavior
        
        x = self.embed(x)
        output = self.output(x)
        output_log = self.LogSof(output)
        
        return output_log

### Training

Below is our training loop, and I recommend that you train on GPU, if available.

**Note that, because we applied a softmax function to our model output, we are using NLLLoss** as opposed to cross entropy. This is because Softmax  in combination with NLLLoss = CrossEntropy loss .

In [ ]:
# check if GPU is available
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = "cpu"
# device = "cpu"
embedding_dim=300 # you can change, if you want

model = SkipGram(len(vocab_to_int), embedding_dim).to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

print_every = 1
steps = 0
epochs = 5

# train for some number of epochs
for e in range(epochs):
    
    # get input and target batches
    for inputs, targets in get_batches(train_words, 512):
        steps += 1
        inputs, targets = torch.LongTensor(inputs), torch.LongTensor(targets)
        inputs, targets = inputs.to(device), targets.to(device)
        
        log_ps = model(inputs)
        loss = criterion(log_ps, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if steps % print_every == 0:                  
            # getting examples and similarities      
            valid_examples, valid_similarities = cosine_similarity(model.embed, device=device)
            _, closest_idxs = valid_similarities.topk(6) # topk highest similarities
            
            valid_examples, closest_idxs = valid_examples.to('cpu'), closest_idxs.to('cpu')
            for ii, valid_idx in enumerate(valid_examples):
                closest_words = [int_to_vocab[idx.item()] for idx in closest_idxs[ii]][1:]
                print(int_to_vocab[valid_idx.item()] + " | " + ', '.join(closest_words))
            print("...")

can | glide, beale, ethically, racer, rodrigues
some | succeded, aris, bubba, shrinks, atp
for | monomers, drop, federation, fontane, dhea
seven | miscellany, treviso, culpable, pawl, boz
and | baraka, mtp, sup, aktion, conjectural
be | codon, duodenal, libertad, seth, cocaine
as | configurations, removable, cutscenes, developer, question
system | gemayel, irrealism, dorchester, pbsuccess, francisci
heavy | gcc, hew, welling, brahmaputra, mountaineer
assembly | bosco, doped, ejection, categorised, skier
ice | chemokine, niigata, umi, croft, credentes
experience | rappers, ossie, fanzines, labor, erlewine
liberal | pathogen, wasn, wardens, carinae, repeat
engineering | mosquitoes, ryaku, restored, mismatched, entertainments
orthodox | highlighted, hose, loophole, prizes, shi
applications | zimbabwean, wexford, andover, divan, brutality
...
who | rediscovered, imam, medicated, palisades, boring
d | cotten, platforms, insider, voris, overzealous
history | clemson, gleam, postfix, cosmic, 

when | heterotic, mayer, saragossa, scraps, akshara
may | email, meltwater, dominican, travelers, courtenay
over | genetically, stram, rudin, janet, basques
use | kuna, meaningful, orleans, feyd, toughened
american | heck, telekinetic, burrowing, parva, stitch
five | ortiz, thurgood, kaufman, linking, agm
only | supervised, izzard, labored, bureaucracy, pareto
for | monomers, drop, federation, fontane, dhea
cost | hydraulic, anguage, verbal, protestantism, minced
units | scant, assertion, purposefully, platonist, aboard
police | dependable, uniao, caravan, stoke, whomever
engineering | mosquitoes, ryaku, restored, mismatched, entertainments
channel | relationship, jackson, superceded, rukh, deacon
centre | imo, athelstan, hedonist, extend, forties
heavy | gcc, hew, welling, brahmaputra, mountaineer
stage | stalks, milliardo, catechism, etheridge, pvc
...
at | cristina, fiestas, tendon, montmorency, secure
who | rediscovered, imam, medicated, palisades, boring
are | exported, duns, coro

time | subservience, indymedia, cerium, fishery, kidnapped
first | frag, laments, dim, amoeboids, rice
on | bogor, cracker, continent, katharine, giulio
have | wrote, veda, longbows, lonsdale, flynn
up | secret, antinori, mystique, conformation, omer
zero | reddish, discrediting, lahn, oppressors, izabella
more | codex, coleco, rounders, serialisation, graphein
war | siva, bertolt, meda, fatalism, igs
operations | polymorphic, millennialism, cosby, condensing, playboys
channel | relationship, jackson, superceded, rukh, deacon
resources | hercule, cordova, norah, tors, parenting
animals | drags, alleviation, dost, whitehaven, bleaches
egypt | langdon, pulaski, surprisingly, aimed, marketplace
san | carn, fate, wally, secede, rearward
placed | debuts, pascha, elo, microorganism, mutinous
rise | personally, freeh, brokerage, dix, thereabouts
...
in | buailte, alpenhorn, dogbert, intellects, clipped
with | marple, pale, artha, attracted, gaborone
was | mismanagement, nahin, aural, suitors,

world | intensities, heels, biomes, monteverdi, sharps
when | mayer, heterotic, scraps, saragossa, akshara
where | gallardo, grudging, jonesboro, gutar, whitby
can | glide, beale, ethically, racer, rodrigues
however | elevates, bowman, enlarged, moneo, heroine
many | musicologists, agoraphobic, dvo, eliminating, embodied
will | lint, fabrications, berkelium, bessus, interconnects
would | junkers, lockyer, ying, individually, competent
channel | relationship, jackson, superceded, rukh, deacon
institute | packed, tallies, complicating, mohandas, sympathetically
ice | chemokine, niigata, umi, croft, credentes
universe | russification, gradation, neorealism, manufacturer, unconsciousness
proposed | reinsurance, corkscrew, robin, grozny, son
notes | hamlin, lynx, obadiah, monologues, clash
liberal | pathogen, wasn, wardens, carinae, hoppers
alternative | extant, leech, samples, expensive, tisza
...
been | microstate, jenny, gallic, oleg, clamp
world | intensities, heels, biomes, monteverdi,

th | overdose, gumbo, archbishopric, memeplex, droit
american | heck, telekinetic, burrowing, parva, stitch
often | hypothalamus, hutchence, drones, approbation, apis
some | succeded, aris, bubba, shrinks, atp
would | junkers, lockyer, ying, individually, competent
the | woody, gilmore, poz, campylobacter, longitudes
of | coles, caver, waving, crucis, aorist
most | bowdoin, toughness, chronicler, ch, phane
magazine | invites, breton, restructure, hector, eritrean
liberal | pathogen, wasn, wardens, carinae, repeat
powers | imperfections, suchet, radek, wumpus, worships
question | marginalism, europeans, delete, tattooed, as
square | geo, approve, plowman, intimate, kristian
older | fal, jass, lapses, bikinis, cosmopolitan
grand | algerian, revisit, jared, smallest, impaled
behind | josh, dieting, exploitable, pleated, gatehouse
...
years | perpetuating, incunabula, ivorian, drains, sponsored
their | winnie, trend, faux, achaemenid, generalizes
up | secret, antinori, mystique, omer, conf

...

for | drop, federation, planned, monomers, fontane
united | pythagoreans, nabba, lascaris, dup, farmer
this | chondrites, henry, expedite, bikes, donovan
zero | reddish, oppressors, lahn, discrediting, izabella
known | chancellor, northcote, stabilizer, radicalism, antiretroviral
were | rosemont, balder, hypercard, interpretive, overly
from | magnesium, hawkins, oshima, madhyamaka, vocations
see | observances, evoke, mechanization, dodging, phosphorus
test | fiddlers, cortez, opportunistic, antiwar, fanfare
egypt | langdon, surprisingly, pulaski, aimed, encyclopedist
animals | drags, alleviation, dost, whitehaven, however
bbc | madame, rebar, mechanization, mouch, india
rise | personally, dix, freeh, brokerage, thereabouts
know | epicurean, latins, priestess, mercator, irredentism
marriage | ruthven, antibody, cartoons, ludus, amounts
governor | szczecin, anabolism, magnetometer, montmartre, infer
...
or | kinship, pharisee, brewery, tallis, stevens
their | winnie, faux, achaemenid, tr

their | winnie, faux, achaemenid, trend, generalizes
th | overdose, archbishopric, gumbo, memeplex, tellers
after | descendents, advisors, yakovlev, perishes, monteverdi
they | fatimids, toilet, ean, associations, intense
is | imprimerie, sven, strikers, mississippi, wyche
on | bogor, katharine, cracker, levantine, continent
would | junkers, individually, lockyer, ying, competent
i | cmyk, manet, terms, apologise, habit
pressure | redfield, highlander, html, malloum, pavia
arts | survivals, groomed, plot, beresford, fickle
experience | ossie, rappers, su, labor, fanzines
applied | representations, ural, packaging, conscientious, opus
joseph | weakens, colliding, looks, self, phishing
woman | vn, roosevelt, antonym, fellowships, chiefly
animals | drags, alleviation, dost, whitehaven, however
prince | hagi, livers, heist, stalls, inborn
...
use | kuna, meaningful, orleans, toughened, feyd
on | bogor, katharine, cracker, levantine, giulio
are | exported, duns, coronado, predicts, easley
n

has | pull, incessant, remember, mcduff, observe
he | calcio, dantzig, spherically, rossington, brutish
with | marple, attracted, pale, artha, gaborone
system | gemayel, irrealism, contested, dorchester, pbsuccess
for | drop, federation, planned, fontane, monomers
on | bogor, katharine, cracker, levantine, giulio
no | drawbar, nicolas, paths, murder, gaylord
not | deprogramming, homeward, dispersive, artemis, eradicate
applied | representations, ural, packaging, conscientious, opus
grand | algerian, revisit, jared, impaled, smallest
scale | leftism, revolution, boingo, eddies, shuttlecock
experience | rappers, ossie, su, labor, fanzines
proposed | reinsurance, corkscrew, robin, grozny, gregory
hit | idempotents, iguana, landowner, underbrace, weisstein
liberal | wasn, pathogen, wardens, repeat, barbarians
engineering | mosquitoes, ryaku, restored, mismatched, entertainments
...
for | drop, federation, planned, fontane, monomers
its | oswaldo, commuted, gabba, peasantry, distracted
and 

all | impinging, civ, calories, equanimity, tries
zero | reddish, oppressors, discrediting, lahn, izabella
with | marple, pale, attracted, artha, gaborone
they | fatimids, toilet, associations, ean, pretended
were | rosemont, balder, hypercard, interpretive, overly
united | pythagoreans, nabba, lascaris, dup, farmer
five | ortiz, spondees, agm, kaufman, joanne
see | observances, evoke, mechanization, dodging, phosphorus
versions | justifiable, reappraisal, toilets, iigs, dissolution
test | cortez, fiddlers, opportunistic, antiwar, fanfare
issue | jeopardized, kalmykia, niches, lingus, coquitlam
frac | dieterich, latifolia, chlorine, bala, signalled
bill | refractory, yogas, armoured, bindi, asteraceae
grand | algerian, revisit, jared, impaled, smallest
additional | seebeck, conceding, judge, supergravity, consolation
units | scant, purposefully, assertion, aboard, platonist
...
d | cotten, platforms, insider, overzealous, voris
has | pull, incessant, remember, mcduff, observe
however |

war | siva, bertolt, fatalism, meda, igs
which | coats, reversibility, hawkeye, pune, moralities
not | deprogramming, homeward, dispersive, eradicate, isabella
that | instilled, sealing, sumitomo, livery, dispersing
than | illustration, pleasantville, reuel, buffaloes, instrumentals
was | mismanagement, nahin, affricate, aural, sinful
eight | outs, conceptual, condita, icing, dontrelle
first | laments, frag, amoeboids, undermine, rice
liberal | wasn, pathogen, wardens, repeat, barbarians
assembly | bosco, doped, categorised, skier, exert
lived | migrated, standardise, bethany, month, gradually
dr | hesychasts, lamont, reveal, mercia, heritability
ocean | gramophone, cherub, gildas, tegmark, kli
powers | imperfections, suchet, radek, wumpus, bees
nobel | crannogs, bonobo, agrarianism, monochrome, rassemblement
test | cortez, fiddlers, opportunistic, antiwar, fanfare
...


In [ ]:
# torch.cuda.is_available()

## Visualizing the word vectors

Below we'll use T-SNE to visualize how our high-dimensional word vectors cluster together. T-SNE is used to project these vectors into two dimensions while preserving local stucture. Check out [this post from Christopher Olah](http://colah.github.io/posts/2014-10-Visualizing-MNIST/) to learn more about T-SNE and other ways to visualize high-dimensional data.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
# getting embeddings from the embedding layer of our model, by name
embeddings = model.embed.weight.to('cpu').data.numpy()

In [ ]:
viz_words = 600
tsne = TSNE()
embed_tsne = tsne.fit_transform(embeddings[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(figsize=(16, 16))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)